In [1]:
from pathlib import Path

from pyarrow import parquet as pq

from arcpy_parquet.utils.pyarrow_utils import *

In [24]:
pqt_pth = Path(r"D:\projects\arcpy-parquet\data\sample\main_fgdb_sample\parquet")

In [ ]:
import pyarrow.dataset as ds

# Load the dataset with partitioning specified
dataset = ds.dataset(pqt_pth, format="parquet", partitioning="hive")

# Get the fragments (individual files)
fragments = dataset.get_fragments()

In [56]:
dataset = pq.ParquetDataset(pqt_pth)
fragments = dataset.fragments

In [57]:
# Extract partition expressions and convert to dictionaries
partition_set = set()

# add all the partition expressions as strings to a set to get unique values
for fragment in fragments:
    expr = fragment.partition_expression
    # Convert expression to string and parse manually
    # Example: (year == 2020) → {'year': 2020}
    expr_str = str(expr)
    partition_set.add(expr_str)


partition_lst = []

# parse these out into the discrete parts
for expr_str in partition_set:
    parts = expr_str.replace('(', '').replace(')', '').split(" and ")
    partition = {}
    for part in parts:
        key, _, value = part.partition(" == ")
        partition[key.replace('(', '').replace(')', '').strip()] = eval(value.replace('(', '').replace(')', '').strip())
    partition_lst.append(partition)

partition_lst

[{'delivery_year': 2025, 'delivery_month': 6},
 {'delivery_year': 2024, 'delivery_month': 8},
 {'delivery_year': 2025, 'delivery_month': 8}]

In [58]:
def generate_combinations(dict_list):
    # Extract all keys from the first dictionary (assuming consistent keys)
    keys = list(dict_list[0].keys())
    
    # Create combinations of keys from depth 1 to full depth
    key_combinations = []
    for i in range(1, len(keys) + 1):
        key_combinations.extend([keys[:i]])

    # Use a set to avoid duplicate dictionaries
    result_set = set()
    result = []

    for key_combo in key_combinations:
        for d in dict_list:
            new_dict = {k: d[k] for k in key_combo}
            # Convert to tuple of items to make it hashable
            dict_tuple = tuple(new_dict.items())
            if dict_tuple not in result_set:
                result_set.add(dict_tuple)
                result.append(new_dict)

    return result


generate_combinations(partition_lst)

[{'delivery_year': 2025},
 {'delivery_year': 2024},
 {'delivery_year': 2025, 'delivery_month': 6},
 {'delivery_year': 2024, 'delivery_month': 8},
 {'delivery_year': 2025, 'delivery_month': 8}]